In [ ]:
#| default_exp datasources.mosaico
%load_ext autoreload
%autoreload 2

import sys,os
from pathlib import Path

In [ ]:
# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))
os.chdir(Path.cwd().parent / 'extracao')

# Mosaico
> Módulo para encapsular a extração e processamento comum às bases do Mosaico

In [ ]:
#| export
import os

import pandas as pd
from dotenv import find_dotenv, load_dotenv
from fastcore.foundation import GetAttr
from tqdm.auto import tqdm

from extracao.datasources.connectors import MongoDB
from extracao.datasources.base import Base

In [ ]:
#| export
load_dotenv(find_dotenv())

True

In [ ]:
#| hide: true
#| eval:false
__file__ = Path.cwd().parent / 'extracao' / 'datasources.py'

In [ ]:
#| export
MONGO_URI: str = os.environ.get('MONGO_URI')


In [ ]:
# | export
class Mosaico(Base, GetAttr):
	def __init__(self, mongo_uri: str = MONGO_URI):
		self.database = 'sms'
		self.default = MongoDB(mongo_uri)

	@property
	def collection(self):
		raise NotImplementedError("Subclasses devem implementar a propriedade 'collection'")

	@property
	def query(self):
		raise NotImplementedError("Subclasses devem implementar a propriedade 'query'")

	@property
	def projection(self):
		raise NotImplementedError("Subclasses devem implementar a propriedade 'projection'")

	def _extract(self, collection: str, pipeline: list):
		client = self.connect()
		database = client[self.database]
		collection = database[collection]
		dtype = 'string[pyarrow]' if self.stem == 'srd' else 'category'
		df = pd.DataFrame(list(collection.aggregate(pipeline)), copy=False, dtype=dtype)
		# Substitui strings vazias e somente com espaços por nulo
		return df.replace(r'^\s*$', pd.NA, regex=True)

	def split_designacao(
		self,
		df: pd.DataFrame,  # DataFrame com coluna original DesignacaoEmissao
	) -> pd.DataFrame:  # DataFrame com novas colunas Largura_Emissão(kHz) e Classe_Emissão
		"""Parse a bandwidth string
		It returns the numerical component and a character class
		"""
		df['Designação_Emissão'] = (
			df['Designação_Emissão'].str.replace(',', ' ').str.strip().str.upper().str.split(' ')
		)
		df = df.explode('Designação_Emissão').reset_index(drop=True)
		exploded_rows = df['Designação_Emissão'].apply(lambda x: isinstance(x, list))
		log = """[("Colunas", "Designação_Emissão"]),
		          ("Processamento", "Registro expandido nos componentes individuais e extraídas Largura e Classe")]"""
		df = self.register_log(df, log, exploded_rows)
		df = df[df['Designação_Emissão'] != '/']  # Removes empty rows
		# Apply the parse_bw function
		parsed_data = zip(*df['Designação_Emissão'].apply(Base.parse_bw))
		df['Largura_Emissão(kHz)'], df['Classe_Emissão'] = parsed_data
		return df.drop('Designação_Emissão', axis=1)

In [ ]:
#| hide
from nbdev.doclinks import nbdev_export 
nbdev_export()